In [1]:
%load_ext autoreload
%autoreload

import pathlib
import lam4vb3.build as build
import pandas as pd
import lam4vb3.property_build as pbuild

INPUT_FILE = pathlib.Path("../docs/semi-structured/LAM_metadata_03.xlsx").resolve()
OUTPUT_FILE = pathlib.Path("../output/lam_project_v1.ttl").resolve()

In [2]:
lam_df_properties = pd.read_excel(INPUT_FILE, sheet_name=0 , header = [0], )
lam_df_classes = pd.read_excel(INPUT_FILE, sheet_name=1 ,header = [0], )
celex_df_properties = pd.read_excel(INPUT_FILE, sheet_name=2 ,header = [0], )
celex_df_classes = pd.read_excel(INPUT_FILE, sheet_name=3 ,header = [0], )

# test = lam_df_properties.head().to_dict()
# print(str(test))

In [98]:
# graph = build.make_lam_graph()

# pbuild.create_cs(graph)
# pbuild.create_concepts(lam_df_properties, graph)

# graph.serialize(str(OUTPUT_FILE), format='turtle',)

import numpy as np
import re

def clean_list(lst,delimiters="|~"):
    return [str.strip(re.sub(delimiters,"",x)) for x in lst if x]

# cells with unconsytrained values
# cells with constrained values by conventions in the property sheet

# property_re = re.compile(r"(None|nan)|([\w:\\\/]+(?:\n|\s)*)")
property_re = re.compile(r"[\n]+")


cells_constrained_prop=[None, np.nan, "", "literal", "literal_line \nnew_line_literal",
                                               "skos:prefLabel",
                                               "skos:prefLabel \n skos:altLabel",
                                               "skos:prefLabel \n skos:altLabel ,\nskos:prefLabel\n ,skos:altLabel\nskos:prefLabel \nskos:altLabel",
                                                "Amendment to, \nEarlier related instruments",
                                                "Amendment to \nEarlier related instruments",
                                               ]
for i in cells_constrained_prop:
    s = [str.strip(x) for x in property_re.split(str(i)) if x]
#     s = clean_list(property_re.split(str(i)),delimiters=",")
#     print(s)
    
# cells with constrained values by conventions in the class sheet
class_re = re.compile(r"((?![|~]).+)([|~].+)|(.+)")
value_separator = re.compile(r"\n")
# 

cells_constrained_class = ["", "Y", "Y     ", "N  ", "YU", "OU", "O", "N",
                                            "Y | some comment", "N | some comment", " YU ~ some comment", "yes,",
                                            "according to text", "no", "value", "value | with a comment",
                                            "value ~ with another comment", "value1 \n value2",
                                            "value1 \n value2 | comment",
                                            "skos:prefLabel \n skos:altLabel | comment", "value1\nvalue2"
                                            ]

# controlled list = (none|nan|y|yes|yu|o|ou|n|no)

for i in cells_constrained_class:
    lines = [str.strip(x) for x in re.split(r"\n", i) if x]
    for line in lines:
        
#         parts = re.split(r"[~\|]", line)# [str.strip(x) for x in re.split(r"|~", line) if x]
#         print (":",parts)
        s = [str.strip(x) for x in re.split("[~\|]",line) if x]
        print (s)
        s0 = s[0] if s else None
        s1 = s[1] if len(s)>1 else None
#         s2 = s[1] if len(s)>2 else None
        print (s0,s1)
#     print (lines)


['Y']
Y None
['Y']
Y None
['N']
N None
['YU']
YU None
['OU']
OU None
['O']
O None
['N']
N None
['Y', 'some comment']
Y some comment
['N', 'some comment']
N some comment
['YU', 'some comment']
YU some comment
['yes,']
yes, None
['according to text']
according to text None
['no']
no None
['value']
value None
['value', 'with a comment']
value with a comment
['value', 'with another comment']
value with another comment
['value1']
value1 None
['value2']
value2 None
['value1']
value1 None
['value2', 'comment']
value2 comment
['skos:prefLabel']
skos:prefLabel None
['skos:altLabel', 'comment']
skos:altLabel comment
['value1']
value1 None
['value2']
value2 None
